In [1]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from scipy.stats import spearmanr

## 데이터 읽어오기

In [2]:
ROOT_DIR = "data"
RANDOM_STATE = 110

# Load data
train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
train_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,NaN,NaN,...,7,NaN,NaN,127,NaN,NaN,1,NaN,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,NaN,NaN,...,185,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,NaN,NaN,...,10,NaN,NaN,73,NaN,NaN,1,NaN,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,NaN,NaN,...,268,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,NaN,NaN,...,121,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,NaN,NaN,...,318,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,NaN,NaN,...,14,NaN,NaN,197,NaN,NaN,1,NaN,NaN,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,NaN,NaN,...,1,NaN,NaN,27,NaN,NaN,1,NaN,NaN,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,NaN,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal


In [3]:
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))
test_data

,Set ID,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,0001be084fbc4aaa9d921f39e595961b,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,1,OK,1000.0,NaN,...,195,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN
1,0005bbd180064abd99e63f9ed3e1ac80,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,1,OK,1000.0,NaN,...,14,NaN,NaN,256,NaN,NaN,1,NaN,NaN,NaN
2,000948934c4140d883d670adcb609584,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,1,OK,240.0,NaN,...,98,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN
3,000a6bfd02874c6296dc7b2e9c5678a7,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,1,OK,1000.0,NaN,...,14,NaN,NaN,0,NaN,NaN,1,NaN,NaN,NaN
4,0018e78ce91343678716e2ea27a51c95,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,1,OK,240.0,NaN,...,1,NaN,NaN,215,NaN,NaN,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,1,OK,1000.0,NaN,...,14,NaN,NaN,131,NaN,NaN,1,NaN,NaN,NaN
17357,ffed8923c8a448a98afc641b770be153,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,1,OK,1000.0,NaN,...,12,NaN,NaN,279,NaN,NaN,1,NaN,NaN,NaN
17358,fff1e73734da40adbe805359b3efb462,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,1,OK,240.0,NaN,...,4,NaN,NaN,66,NaN,NaN,1,NaN,NaN,NaN
17359,fff8e38bdd09470baf95f71e92075dec,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,1,OK,240.0,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN


## NULL 값의 칼럼 삭제

In [4]:
train_data = train_data.dropna(axis=1, how='all')
test_data = test_data.dropna(axis=1, how='all')

## 모든 데이터 동일한 칼럼 삭제

In [5]:
nunique_train = train_data.nunique()
nunique_test = test_data.nunique()

### 모든 값이 동일한 칼럼 찾고 삭제

In [6]:
same_value_columns_train = nunique_train[nunique_train == 1].index
same_value_columns_test = nunique_test[nunique_test == 1].index
train_data = train_data.drop(columns=same_value_columns_train)
test_data = test_data.drop(columns=same_value_columns_test)

## 완전히 동일한 칼럼 삭제

In [7]:
train_data_T = train_data.T
test_data_T = test_data.T

In [8]:
duplicate_columns_train = train_data_T.duplicated(keep='first')
duplicate_columns_test = test_data_T.duplicated(keep='first')

In [9]:
train_data = train_data.loc[:, ~duplicate_columns_train]
test_data = test_data.loc[:, ~duplicate_columns_test]

In [10]:
train_data.shape, test_data.shape

((40506, 124), (17361, 123))

#### 이렇게 해도 124, 123개의 칼럼이 존재하므로 상관계수를 통해 데이터를 조금 더 전처리하겠음

### 상관계수 계산 (object 타입은 일단 계산에서 제외했습니다)

In [11]:
corr_with_target = []
p_values = []

In [12]:
numeric_cols = train_data.select_dtypes(include=['float64', 'int64']).columns

for col in numeric_cols:
    corr, p_value = spearmanr(train_data[col], train_data['target'])
    corr_with_target.append(corr)
    p_values.append(p_value)

### 결과를 dataframe으로

In [13]:
corr_df = pd.DataFrame({
    'Column': numeric_cols,
    'Correlation': corr_with_target,
    'p-value': p_values
})

### p-value가 0.05 이상인 칼럼은 제외

In [14]:
low_corr_columns = corr_df[corr_df['p-value'] > 0.05]['Column']

### 상관계수가 낮은 칼럼도 제외

In [15]:
train_data = train_data.drop(columns=low_corr_columns)
test_data = test_data.drop(columns=low_corr_columns, errors='ignore')  # test 데이터셋에 없는 경우가 있을 수 있음

In [16]:
train_data.shape, test_data.shape, low_corr_columns.tolist()[:10]

((40506, 107),
 (17361, 106),
 ['CURE END POSITION X Collect Result_Dam',
  'CURE END POSITION Z Collect Result_Dam',
  'CURE END POSITION Θ Collect Result_Dam',
  'CURE START POSITION X Collect Result_Dam',
  'THICKNESS 2 Collect Result_Dam',
  'THICKNESS 3 Collect Result_Dam',
  '1st Pressure Collect Result_AutoClave',
  '1st Pressure 1st Pressure Unit Time_AutoClave',
  '2nd Pressure Collect Result_AutoClave',
  'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1'])

#### 칼럼을 최대 107, 106개까지 줄였음

### 문자형 데이터만 남아있는 칼럼 처리

In [17]:
def is_all_string(series):
    return series.apply(lambda x: isinstance(x, str)).all()

string_columns = [col for col in train_data.columns if is_all_string(train_data[col])]

print(string_columns)

['Equipment_Dam', 'Model.Suffix_Dam', 'Workorder_Dam', 'Chamber Temp. Judge Value_AutoClave', 'Equipment_Fill1', 'Equipment_Fill2', 'target']


In [18]:
train_data[string_columns]

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Chamber Temp. Judge Value_AutoClave,Equipment_Fill1,Equipment_Fill2,target
0,Dam dispenser #1,AJX75334505,4F1XA938-1,OK,Fill1 dispenser #1,Fill2 dispenser #1,Normal
1,Dam dispenser #1,AJX75334505,3KPM0016-2,OK,Fill1 dispenser #1,Fill2 dispenser #1,Normal
2,Dam dispenser #2,AJX75334501,4E1X9167-1,OK,Fill1 dispenser #2,Fill2 dispenser #2,Normal
3,Dam dispenser #2,AJX75334501,3K1X0057-1,OK,Fill1 dispenser #2,Fill2 dispenser #2,Normal
4,Dam dispenser #1,AJX75334501,3HPM0007-1,NG,Fill1 dispenser #1,Fill2 dispenser #1,Normal
...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334501,3J1XF434-2,OK,Fill1 dispenser #1,Fill2 dispenser #1,Normal
40502,Dam dispenser #2,AJX75334501,4E1XC796-1,NG,Fill1 dispenser #2,Fill2 dispenser #2,Normal
40503,Dam dispenser #1,AJX75334501,4C1XD438-1,NG,Fill1 dispenser #1,Fill2 dispenser #1,Normal
40504,Dam dispenser #2,AJX75334501,3I1XA258-1,OK,Fill1 dispenser #2,Fill2 dispenser #2,Normal


In [19]:
col1 = "Equipment_Dam"
train_data[col1].value_counts()

Equipment_Dam
Dam dispenser #1    25030
Dam dispenser #2    15476
Name: count, dtype: int64

In [20]:
def process_head_normal1(data):
    if data == "Dam dispenser #1":
        return "1"
    elif data == "Dam dispenser #2":
        return "2"


train_data[col1] = train_data[col1].apply(process_head_normal1)
test_data[col1] = test_data[col1].apply(process_head_normal1)

In [21]:
train_data[col1].value_counts()

Equipment_Dam
1    25030
2    15476
Name: count, dtype: int64

In [22]:
col2 = "Model.Suffix_Dam"
train_data[col2].value_counts()

Model.Suffix_Dam
AJX75334501    33820
AJX75334502     3390
AJX75334505     2635
AJX75334507      310
AJX75334503      162
AJX75334506      129
AJX75334508       60
Name: count, dtype: int64

In [23]:
def process_head_normal2(data):
    if data == "AJX75334501":
        return "1"
    elif data == "AJX75334502":
        return "2"
    elif data == "AJX75334503":
        return "3"
    elif data == "AJX75334505":
        return "5"
    elif data == "AJX75334506":
        return "6"
    elif data == "AJX75334507":
        return "7"
    elif data == "AJX75334508":
        return "8"
    
train_data[col2] = train_data[col2].apply(process_head_normal2)
test_data[col2] = test_data[col2].apply(process_head_normal2)

In [24]:
train_data[col2].value_counts()

Model.Suffix_Dam
1    33820
2     3390
5     2635
7      310
3      162
6      129
8       60
Name: count, dtype: int64

In [25]:
col3 = "Workorder_Dam"
value_counts3 = train_data[col3].value_counts()
value_counts33 = test_data[col3].value_counts()

# 2. 빈도에 따라 고유 값을 정렬하고, 순서에 따라 문자 '1', '2', '3' 등으로 매핑
mapping3 = {value: str(index + 1) for index, value in enumerate(value_counts3.index)}
mapping33 = {value: str(index + 1) for index, value in enumerate(value_counts33.index)}

# 3. 데이터프레임의 칼럼을 매핑된 값으로 업데이트
train_data[col3] = train_data[col3].map(mapping3)
test_data[col3] = test_data[col3].map(mapping33)

print(train_data[col3])

0        103
1        240
2         95
3         94
4        461
        ... 
40501    471
40502    265
40503    221
40504     54
40505     61
Name: Workorder_Dam, Length: 40506, dtype: object


In [26]:
col4 = "Equipment_Fill1"
train_data[col4].value_counts()

Equipment_Fill1
Fill1 dispenser #1    25027
Fill1 dispenser #2    15479
Name: count, dtype: int64

In [27]:
value_counts4 = train_data[col4].value_counts()
value_counts44 = test_data[col4].value_counts()

# 2. 빈도에 따라 고유 값을 정렬하고, 순서에 따라 문자 '1', '2', '3' 등으로 매핑
mapping4 = {value: str(index + 1) for index, value in enumerate(value_counts4.index)}
mapping44 = {value: str(index + 1) for index, value in enumerate(value_counts44.index)}

# 3. 데이터프레임의 칼럼을 매핑된 값으로 업데이트
train_data[col4] = train_data[col4].map(mapping4)
test_data[col4] = test_data[col4].map(mapping44)

print(train_data[col4])

0        1
1        1
2        2
3        2
4        1
        ..
40501    1
40502    2
40503    1
40504    2
40505    1
Name: Equipment_Fill1, Length: 40506, dtype: object


In [28]:
train_data[col4].value_counts()

Equipment_Fill1
1    25027
2    15479
Name: count, dtype: int64

#### HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam 컬럼의 NaN 값과 숫자형 데이터를 0로 변경, OK 값은 1로.

In [29]:
column_to_process = "HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam"

def process_head_normal(data):
    if data == "OK":
        return "1"
    else:
        return "0"


train_data[column_to_process] = train_data[column_to_process].apply(process_head_normal)
test_data[column_to_process] = test_data[column_to_process].apply(process_head_normal)

In [30]:
train_data[column_to_process]

0        0
1        0
2        1
3        0
4        0
        ..
40501    0
40502    0
40503    1
40504    0
40505    0
Name: HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam, Length: 40506, dtype: object

In [31]:
train_data[column_to_process].value_counts()

HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam
0    29213
1    11293
Name: count, dtype: int64

In [32]:
col5 = "Equipment_Fill2"
train_data[col5].value_counts()

Equipment_Fill2
Fill2 dispenser #1    25026
Fill2 dispenser #2    15480
Name: count, dtype: int64

In [33]:
value_counts5 = train_data[col5].value_counts()
value_counts55 = test_data[col5].value_counts()

# 2. 빈도에 따라 고유 값을 정렬하고, 순서에 따라 문자 '1', '2', '3' 등으로 매핑
mapping5 = {value: str(index + 1) for index, value in enumerate(value_counts5.index)}
mapping55 = {value: str(index + 1) for index, value in enumerate(value_counts55.index)}

# 3. 데이터프레임의 칼럼을 매핑된 값으로 업데이트
train_data[col5] = train_data[col5].map(mapping5)
test_data[col5] = test_data[col5].map(mapping55)

print(train_data[col5])

0        1
1        1
2        2
3        2
4        1
        ..
40501    1
40502    2
40503    1
40504    2
40505    1
Name: Equipment_Fill2, Length: 40506, dtype: object


### Normal을 1, AbNormal을 0으로

In [34]:
col6 = "Chamber Temp. Judge Value_AutoClave"

def process_head_normal6(data):
    if data == "OK":
        return "1"
    elif data == "NG":
        return "0"


train_data[col6] = train_data[col6].apply(process_head_normal6)
test_data[col6] = test_data[col6].apply(process_head_normal6)

In [35]:
train_data[string_columns]

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Chamber Temp. Judge Value_AutoClave,Equipment_Fill1,Equipment_Fill2,target
0,1,5,103,1,1,1,Normal
1,1,5,240,1,1,1,Normal
2,2,1,95,1,2,2,Normal
3,2,1,94,1,2,2,Normal
4,1,1,461,0,1,1,Normal
...,...,...,...,...,...,...,...
40501,1,1,471,1,1,1,Normal
40502,2,1,265,0,2,2,Normal
40503,1,1,221,0,1,1,Normal
40504,2,1,54,1,2,2,Normal


In [36]:
string_columns_test_before = [col for col in test_data.columns if is_all_string(test_data[col])]
test_data[string_columns_test_before]

,Set ID,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,Chamber Temp. Judge Value_AutoClave,Equipment_Fill1,Equipment_Fill2
0,0001be084fbc4aaa9d921f39e595961b,2,1,154,0,1,2,2
1,0005bbd180064abd99e63f9ed3e1ac80,2,1,549,1,1,2,2
2,000948934c4140d883d670adcb609584,1,1,25,0,1,1,1
3,000a6bfd02874c6296dc7b2e9c5678a7,2,1,48,0,1,2,2
4,0018e78ce91343678716e2ea27a51c95,1,1,357,1,1,1,1
...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,2,1,99,0,1,2,2
17357,ffed8923c8a448a98afc641b770be153,2,1,177,1,1,2,2
17358,fff1e73734da40adbe805359b3efb462,1,1,66,0,1,1,1
17359,fff8e38bdd09470baf95f71e92075dec,1,1,135,0,1,1,1


In [37]:
test_data = test_data.drop(columns="Set ID")

In [38]:
string_columns_test = [col for col in test_data.columns if is_all_string(test_data[col])]
test_data[string_columns_test]

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,Chamber Temp. Judge Value_AutoClave,Equipment_Fill1,Equipment_Fill2
0,2,1,154,0,1,2,2
1,2,1,549,1,1,2,2
2,1,1,25,0,1,1,1
3,2,1,48,0,1,2,2
4,1,1,357,1,1,1,1
...,...,...,...,...,...,...,...
17356,2,1,99,0,1,2,2
17357,2,1,177,1,1,2,2
17358,1,1,66,0,1,1,1
17359,1,1,135,0,1,1,1


### 언더샘플링

In [39]:
df_normal = train_data[train_data["target"] == "Normal"]
df_abnormal = train_data[train_data["target"] == "AbNormal"]

df_normal = df_normal.sample(n=len(df_abnormal), replace=False, random_state=RANDOM_STATE)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)

In [40]:
df_train, df_val = train_test_split(
    df_concat,
#     test_size=0.3,
    test_size=0.3,
    stratify=df_concat["target"],
    random_state=RANDOM_STATE,
)


def print_stats(df: pd.DataFrame):
    num_normal = len(df[df["target"] == "Normal"])
    num_abnormal = len(df[df["target"] == "AbNormal"])

    print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}" + f" ratio: {num_abnormal/num_normal}")


# Print statistics
print(f"  \tAbnormal\tNormal")
print_stats(df_train)
print_stats(df_val)

  	Abnormal	Normal
  Total: Normal: 1645, AbNormal: 1645 ratio: 1.0
  Total: Normal: 705, AbNormal: 705 ratio: 1.0


## 3. 모델 학습


### 모델 정의


In [41]:
model = RandomForestClassifier(random_state=RANDOM_STATE)

### 모델 학습

In [46]:
features = []
for col in df_train.columns:
    try:
        df_train[col] = df_train[col].astype(int)
        features.append(col)
    except:
        continue
        
train_x = df_train[features]
train_y = df_train["target"]
model.fit(train_x, train_y)
train_x

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE SPEED Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage1) Collect Result_Dam,Dispense Volume(Stage2) Collect Result_Dam,...,Head Clean Position X Collect Result_Fill2,Head Clean Position Y Collect Result_Fill2,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2
4417,1,1,104,70,10,9,3,9,0,0,...,119,50,91,270,85,19,2,27,1,0
1383,1,1,196,70,10,9,4,9,0,0,...,119,50,91,270,85,19,4,64,1,0
895,1,1,99,70,10,9,3,9,0,0,...,119,50,91,270,85,18,1,60,1,0
1270,2,1,272,70,16,13,6,13,1,0,...,-10,119,50,91,50,85,19,13,84,1
1002,2,1,90,70,10,21,10,21,1,0,...,-10,119,50,91,50,85,19,16,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3021,1,1,347,70,10,9,4,9,0,0,...,119,50,91,270,85,19,7,75,1,0
2973,1,1,534,70,10,9,4,9,0,0,...,119,50,91,270,85,18,3,249,1,0
4040,2,1,286,70,16,13,8,13,0,0,...,-10,119,50,91,50,85,19,14,1,1
2294,2,1,90,70,10,21,10,21,1,0,...,-10,119,50,91,50,85,19,9,21,1


## 4. 제출하기


### 테스트 데이터 예측


In [47]:
df_test_x = test_data[features]
for col in df_test_x.columns:
    try:
        df_test_x.loc[:, col] = df_test_x[col].astype(int)
    except:
        continue

KeyError: "['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1'] not in index"

In [ ]:
test_pred = model.predict(df_test_x)
test_pred

### 제출 파일 작성


In [ ]:
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = test_pred

In [ ]:
df_sub.to_csv("submission.csv", index=False)

In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**
